# IRENE-PMAPS-MC-KR

#### Prototype, IRENE reconstruction, Kripton MC data

In [ ]:
from __future__ import print_function

In [ ]:
import time
%matplotlib inline
%load_ext autoreload
import matplotlib.pyplot as plt
import pandas as pd
import tables as tb
import numpy as np
import Core.mplFunctions as mpl
import Core.wfmFunctions as wfm
from Database import loadDB
import ICython.Sierpe.BLR as blr
import ICython.Core.peakFunctions as cpf
import Core.peakFunctions as pf
from ICython.Core.system_of_units import SystemOfUnits 
from scipy import signal
%load_ext Cython

In [ ]:
print(time.strftime("%d/%m/%Y"))
print(time.strftime("%H:%M:%S"))

In [ ]:
from time import time

In [ ]:
import Cities.IRENE as irene

### Kripton MC

1. P = 7 bar
2. V_gate = 7kV: E/P = 2
3. Processed by DIOMIRA

### Kripton MC file

In [ ]:
path='/Users/jjgomezcadenas/Documents/Development/NEXT/icdata/MC/'
ffile = 'ic_dst_NEXT_v0_08_06_Kr_ACTIVE_0_0_7bar_MCRD_10000.root.h5'
fkr = tb.open_file(path+ffile,'r+')
pmtrwf = fkr.root.RD.pmtrwf
sipmrwf = fkr.root.RD.sipmrwf
NEVT, NPMT, PMTWL = pmtrwf.shape
NEVT, NSIPM, SIPMWL = sipmrwf.shape
print("""
        Number of events in file = {}
        Number of PMTs = {}
        PMTWL = {}
        Number of SiPMs = {}
        SiPMWL = {}
      """.format(NEVT, NPMT, PMTWL,NSIPM, SIPMWL))

### Access to data base and definition of units

In [ ]:
DataPMT = loadDB.DataPMT()
units = SystemOfUnits()
adc_to_pes = abs(DataPMT.adc_to_pes.values)
coeff_c = abs(DataPMT.coeff_c.values)
coeff_blr = abs(DataPMT.coeff_blr.values)
DataSiPM = loadDB.DataSiPM(2976)
adc_to_pes_sipm = DataSiPM.adc_to_pes.values
xs = DataSiPM.X.values
ys = DataSiPM.Y.values

In [ ]:
DataPMT

### Structure of file

In [ ]:
fkr

In [ ]:
fkr.__contains__('/Run')

In [ ]:
if '/Run' in fkr:
    print('True')
else:
    print('False')


### Plot Raw Waveform

In [ ]:
event = 0
mpl.plot_waveforms(pmtrwf[event])

### Deconvolution

In [ ]:
event = 0
t0 = time()
CWF = blr.deconv_pmt(pmtrwf[event], coeff_c, coeff_blr)
t1 = time()        
dt = t1 - t0

print("run over {} events  in {} s".format(event,dt))
mpl.plot_waveforms(CWF)

### calibrated PMT sum

In [ ]:
event = 0
t0 = time()
CWF = blr.deconv_pmt(pmtrwf[event], coeff_c, coeff_blr)
csum = cpf.calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=100, thr_MAU=3) 
t1 = time()        
dt = t1 - t0

print("run over {} events  in {} s".format(event,dt))

In [ ]:
tstep = 25
signal_t = np.arange(0., PMTWL * tstep, tstep)

In [ ]:
mpl.plot_signal(signal_t/units.mus, csum, title="calibrated sum",
                signal_start=0, signal_end=1200, 
                ymax = 100, 
                t_units='mus', units="pes")

In [ ]:
mpl.plot_signal(signal_t/units.mus, csum, title="calibrated sum",
                signal_start=260, signal_end=280, 
                ymax = 60, 
                t_units='mus', units="pes")

In [ ]:
np.sum(csum)

In [ ]:
mpl.plot_signal(signal_t/units.mus, csum, title="calibrated sum",
                signal_start=99, signal_end=101, 
                ymax = 5, 
                t_units='mus', units="pes")

###  ZS in PMTs

In [ ]:
event = 0
t0 = time()
CWF = blr.deconv_pmt(pmtrwf[event], coeff_c, coeff_blr)
csum = cpf.calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=200, thr_MAU=5) 
wfzs_ene, wfzs_indx = cpf.wfzs(csum, threshold=0.5*units.pes)
wfzs_t = cpf.time_from_index(wfzs_indx)
t1 = time()        
dt = t1 - t0

print("run over {} events  in {} s".format(event,dt))

In [ ]:
mpl.plot_signal(wfzs_t/units.mus, wfzs_ene, title="calibrated sum, ZS",
                signal_start=260, signal_end=280, 
                ymax = 60, 
                t_units='mus', units="pes")

### Find S12 

In [ ]:
event = 0
t0 = time()
CWF = blr.deconv_pmt(pmtrwf[event], coeff_c, coeff_blr)
csum = cpf.calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=100, thr_MAU=3) 
wfzs_ene, wfzs_indx = cpf.wfzs(csum, threshold=0.25*units.pes)
wfzs_t = cpf.time_from_index(wfzs_indx)
S1 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=10,  tmax=110*units.mus, 
                 lmin=5, lmax=20, stride=4,
                 rebin=False)
S2 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=110*units.mus,  tmax=1190*units.mus, 
                 lmin=100, lmax=300, stride=40,
                 rebin=True, rebin_stride=40)
t1 = time()        
dt = t1 - t0

print("run over {} events  in {} s".format(event,dt))

In [ ]:
pf.scan_S12(S1)

In [ ]:
pf.scan_S12(S2)

### SiPMs

In [ ]:
event=0
mpl.plot_sipm(sipmrwf[event], nmin=0, nmax=16, x=4, y=4)

### Subtract baseline, and set a cut to supress dark current

In [ ]:
event=0
t0 = time()
sipm = cpf.signal_sipm(sipmrwf[event], adc_to_pes_sipm, thr=5*units.pes, n_MAU=100)
SIPM = cpf.select_sipm(sipm)
t1 = time()        
dt = t1 - t0

print("run in {} s".format(dt))
print('number of SiPM with signal = {}'.format(len(SIPM)))

In [ ]:
mpl.plot_sipm(sipm, nmin=0, nmax=16, x=4, y=4)

In [ ]:
SIPM

### Selecting SiPMs: threshold on the window 

In [ ]:
def sipm_area_S2(sipml):
    """
    Computes and histgram the area in S2 window for SiPMs selected in sipml
    """
    area_s2 = np.zeros(len(sipml), dtype=np.double)
    for i, sipm in enumerate(sipml):
        area_s2[i] = np.sum(sipm[1])
    return area_s2

### Low threshold

In [ ]:
sipml = pf.sipm_S2(SIPM,S2[0], thr=5*units.pes)

In [ ]:
sipml

In [ ]:
area_s2 = sipm_area_S2(sipml)

In [ ]:
plt.hist(area_s2[area_s2<1000], bins=20)

### A cut at 20-40 separates signal from bkg

In [ ]:
sipmd = pf.sipm_S2_dict(SIPM, S2, thr=40*units.pes)

In [ ]:
sipmd

### The full pre-proc 

In [ ]:
event = 0
t0 = time()
CWF = blr.deconv_pmt(pmtrwf[event], coeff_c, coeff_blr)
csum = cpf.calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=100, thr_MAU=3) 
wfzs_ene, wfzs_indx = cpf.wfzs(csum, threshold=0.25*units.pes)
wfzs_t = cpf.time_from_index(wfzs_indx)
S1 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=10,  tmax=110*units.mus, 
                 lmin=5, lmax=20, stride=4,
                 rebin=False)
S2 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=110*units.mus,  tmax=1190*units.mus, 
                 lmin=100, lmax=300, stride=40,
                 rebin=True, rebin_stride=40)
sipm = cpf.signal_sipm(sipmrwf[event], adc_to_pes_sipm, thr=5*units.pes, n_MAU=100)
SIPM = cpf.select_sipm(sipm)
sipmd = pf.sipm_S2_dict(SIPM, S2, thr=40*units.pes)
t1 = time()        
dt = t1 - t0

print("run in {} s".format(dt))

In [ ]:
def plot_ene_sipm(sipmd, Xs, Ys, radius=3):
    """
    plots the reconstructed energy of the SiPMs
    input: sipm dictionary
    """

    r = np.ones(len(Xs)) * radius
    col = np.zeros(len(Xs))
    for i in sipmd.keys():
        sipml = sipmd[i]
        for sipm in sipml:
            sipm_n = sipm[0]
            sipm_wf = sipm[1]
            sipm_e = np.sum(sipm_wf)
            col[sipm_n] = sipm_e

    plt.figure(figsize=(10, 10))
    plt.subplot(aspect="equal")
    mpl.circles(Xs, Ys, r, c=col, alpha=0.5, ec="none")
    plt.colorbar()

    plt.xlim(-198, 198)
    plt.ylim(-198, 198)
    
    return col

In [ ]:
col = plot_ene_sipm(sipmd,xs,ys)

In [ ]:
plt.plot(col)